In [2]:
import geopandas as gp
import csv
import pandas as pd
from shapely.geometry import LineString
import mysql.connector
from shapely.geometry import Polygon, mapping
import datetime
import json 

In [11]:
file = "./state_shapes.json"
state_boundary = gp.read_file(file)

In [12]:
state_boundary.head()

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,1,2,42,01779798,42,PA,Pennsylvania,00,G4000,A,115880457407,3398574954,+40.9046013,-077.8275298,"MULTIPOLYGON (((-80.51935 41.84956, -80.51922 ..."
1,3,6,21,01779786,21,KY,Kentucky,00,G4000,A,102282218059,2372611005,+37.5336844,-085.2929801,"MULTIPOLYGON (((-88.06133 37.50533, -88.06368 ..."
2,1,2,36,01779796,36,NY,New York,00,G4000,A,122050000805,19246143659,+42.9133974,-075.5962723,"MULTIPOLYGON (((-79.31214 42.68680, -79.14347 ..."


In [14]:
#Rename columns
state_boundary.rename(columns = {"STUSPS":"id"}, inplace = True)

#Drop column
state_boundary = state_boundary.drop (columns = ['REGION', 'DIVISION', 'STATEFP', 'STATENS', 'GEOID', 'NAME', 
                         'LSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON'])

#Add type columns
state_boundary.insert(1, 'centerX', None)
state_boundary.insert(2, 'centerY', None)
state_boundary.insert(3, 'area', None)

state_boundary.head()


,id,centerX,centerY,area,geometry
0,PA,None,None,None,"MULTIPOLYGON (((-80.51935 41.84956, -80.51922 ..."
1,KY,None,None,None,"MULTIPOLYGON (((-88.06133 37.50533, -88.06368 ..."
2,NY,None,None,None,"MULTIPOLYGON (((-79.31214 42.68680, -79.14347 ..."


In [15]:
#ADD area and centers
for i, row in state_boundary.iterrows():
    state_boundary.at[i, 'area'] = row.geometry.area
    bounds = row.geometry.bounds
    centerX = (bounds[0] + bounds[2]) / 2
    centerY = (bounds[1] + bounds[3]) / 2
    state_boundary.at[i, 'centerX'] = centerX
    state_boundary.at[i, 'centerY'] = centerY

state_boundary.head()

,id,centerX,centerY,area,geometry
0,PA,-77.6048,41.1179,12.7472,"MULTIPOLYGON (((-80.51935 41.84956, -80.51922 ..."
1,KY,-85.7696,37.8223,10.6681,"MULTIPOLYGON (((-88.06133 37.50533, -88.06368 ..."
2,NY,-75.77,42.746,15.5804,"MULTIPOLYGON (((-79.31214 42.68680, -79.14347 ..."


In [16]:
state_boundary.to_file("./states_boundary.geojson", driver='GeoJSON')

In [19]:
#POPULATE DATABASE
mydb = mysql.connector.connect(
    host="mysql3.cs.stonybrook.edu",
    user='mhaczynska',
    passwd="110670160",
    database = 'mhaczynska'    
)

cursor = mydb.cursor()
print (mydb)

In [20]:
file = "./states_boundary.geojson"
states = gp.read_file(file)
states.head()


,id,centerX,centerY,area,geometry
0,PA,-77.6048035,41.1179355,12.7471968513065,"MULTIPOLYGON (((-80.51935 41.84956, -80.51922 ..."
1,KY,-85.76960750000001,37.82234,10.668081506716,"MULTIPOLYGON (((-88.06133 37.50533, -88.06368 ..."
2,NY,-75.77002999999999,42.7460485,15.58035125575651,"MULTIPOLYGON (((-79.31214 42.68680, -79.14347 ..."


In [22]:
add_state_querry = ("INSERT INTO mhaczynska.state "
               "(NAME, CENTER_X, CENTER_Y, STATE_GEOJSON) "
               "VALUES (%s, %s, %s, %s)")

#update_precinct_querry = "UPDATE mhaczynska.precinct SET CENTER_X = %s, CENTER_Y = %s WHERE GEO_ID = %s"

for index, row in states.iterrows():
    geometry_str = row.geometry.__geo_interface__
    geometry_str = json.dumps(geometry_str)     
    state_data = (row['id'], row['centerX'], row['centerY'], geometry_str)
    #update_precinct = (row['centerX'], row['centerY'], row['id'])
    cursor.execute(add_state_querry, state_data)
    mydb.commit()

In [23]:
cursor.close()
mydb.close()